In [3]:
from selenium import webdriver
from bs4 import BeautifulSoup
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import pandas as pd
from selenium.webdriver.chrome.options import Options
import pickle
import json
import numpy as np

from IPython.display import clear_output

### 1. 의류 인기순 150개 (상품 번호 리스트에 저장)

In [16]:
options = Options()
driver = webdriver.Chrome(options=options)
full_url = "https://zigzag.kr/categories/-1?title=%EC%9D%98%EB%A5%98&category_id=-1&middle_category_id=-1&sort=200"  # 지그재그 - 의류 - 인기순
driver.get(full_url)

# 페이지 로딩 대기
time.sleep(2)

all_indices = []  # 순서 유지 (set 대신 list 사용)
all_links = []  # 링크도 순서 유지

# 스크롤 실행
scroll_count = 70  # 원하는 만큼 변경 가능
for i in range(scroll_count):
    print(f"{i}번째 스크롤중..")
    driver.execute_script("window.scrollBy(0, 330);")  # 한 화면 아래로 이동
    time.sleep(2)  # 데이터 로딩 대기

    # HTML 가져와서 파싱
    html = driver.page_source
    soup = BeautifulSoup(html, "html.parser")
    
    # data-index 요소 찾기 (중복 방지하면서 순서 유지)
    items = soup.select('[data-index]')
    for item in items:
        index = item.get("data-index")
        if index not in all_indices:  # 중복 방지
            all_indices.append(index)  # 순서 유지하면서 추가

    # 상품 링크 수집 (중복 방지하면서 순서 유지)
    links = soup.select('a.css-152zj1o.product-card-link')
    for link in links:
        href = link.get("href")
        if href and href not in all_links:  # 중복 방지
            all_links.append(href)  # 순서 유지하면서 추가

# # 정렬 X (스크롤한 순서 그대로 출력)
# for index in all_indices:
#     print(f"data-index={index}")

# # 수집한 링크 출력
# print("\n수집한 링크 목록:")
# for link in all_links:
#     print(link)

# 드라이버 종료
driver.quit()


0번째 스크롤중..
1번째 스크롤중..
2번째 스크롤중..
3번째 스크롤중..
4번째 스크롤중..
5번째 스크롤중..
6번째 스크롤중..
7번째 스크롤중..
8번째 스크롤중..
9번째 스크롤중..
10번째 스크롤중..
11번째 스크롤중..
12번째 스크롤중..
13번째 스크롤중..
14번째 스크롤중..
15번째 스크롤중..
16번째 스크롤중..
17번째 스크롤중..
18번째 스크롤중..
19번째 스크롤중..
20번째 스크롤중..
21번째 스크롤중..
22번째 스크롤중..
23번째 스크롤중..
24번째 스크롤중..
25번째 스크롤중..
26번째 스크롤중..
27번째 스크롤중..
28번째 스크롤중..
29번째 스크롤중..
30번째 스크롤중..
31번째 스크롤중..
32번째 스크롤중..
33번째 스크롤중..
34번째 스크롤중..
35번째 스크롤중..
36번째 스크롤중..
37번째 스크롤중..
38번째 스크롤중..
39번째 스크롤중..
40번째 스크롤중..
41번째 스크롤중..
42번째 스크롤중..
43번째 스크롤중..
44번째 스크롤중..
45번째 스크롤중..
46번째 스크롤중..
47번째 스크롤중..
48번째 스크롤중..
49번째 스크롤중..
50번째 스크롤중..
51번째 스크롤중..
52번째 스크롤중..
53번째 스크롤중..
54번째 스크롤중..
55번째 스크롤중..
56번째 스크롤중..
57번째 스크롤중..
58번째 스크롤중..
59번째 스크롤중..
60번째 스크롤중..
61번째 스크롤중..
62번째 스크롤중..
63번째 스크롤중..
64번째 스크롤중..
65번째 스크롤중..
66번째 스크롤중..
67번째 스크롤중..
68번째 스크롤중..
69번째 스크롤중..


In [17]:
len(all_links)

183

In [18]:
의류_links = all_links[:150]
의류_links

['/catalog/products/130108350',
 '/catalog/products/147921597',
 '/catalog/products/113041967',
 '/catalog/products/128943162',
 '/catalog/products/129147932',
 '/catalog/products/113067237',
 '/catalog/products/120508523',
 '/catalog/products/132082569',
 '/catalog/products/159140443',
 '/catalog/products/128434844',
 '/catalog/products/137691186',
 '/catalog/products/111005399',
 '/catalog/products/110294858',
 '/catalog/products/123449569',
 '/catalog/products/141913686',
 '/catalog/products/136660836',
 '/catalog/products/129076367',
 '/catalog/products/109956114',
 '/catalog/products/150012796',
 '/catalog/products/114744900',
 '/catalog/products/114744915',
 '/catalog/products/108649128',
 '/catalog/products/101062215',
 '/catalog/products/137506712',
 '/catalog/products/133446788',
 '/catalog/products/115154949',
 '/catalog/products/135330479',
 '/catalog/products/133185968',
 '/catalog/products/127229360',
 '/catalog/products/127735100',
 '/catalog/products/140288768',
 '/catal

In [ ]:
# 의류 리스트 저장
with open("의류_links.pkl","wb") as f:
    pickle.dump(의류_links, f)

### 2. 의류 상품 정보 가져오기

In [24]:
import pickle
# 불러오기 (링크 가져오는 코드. 이 코드를 돌리면 위에 코드는 다시 안 돌려도 됨)
with open("의류_links.pkl","rb") as f:
    의류_links = pickle.load(f)
    
의류_links

['/catalog/products/130108350',
 '/catalog/products/147921597',
 '/catalog/products/113041967',
 '/catalog/products/128943162',
 '/catalog/products/129147932',
 '/catalog/products/113067237',
 '/catalog/products/120508523',
 '/catalog/products/132082569',
 '/catalog/products/159140443',
 '/catalog/products/128434844',
 '/catalog/products/137691186',
 '/catalog/products/111005399',
 '/catalog/products/110294858',
 '/catalog/products/123449569',
 '/catalog/products/141913686',
 '/catalog/products/136660836',
 '/catalog/products/129076367',
 '/catalog/products/109956114',
 '/catalog/products/150012796',
 '/catalog/products/114744900',
 '/catalog/products/114744915',
 '/catalog/products/108649128',
 '/catalog/products/101062215',
 '/catalog/products/137506712',
 '/catalog/products/133446788',
 '/catalog/products/115154949',
 '/catalog/products/135330479',
 '/catalog/products/133185968',
 '/catalog/products/127229360',
 '/catalog/products/127735100',
 '/catalog/products/140288768',
 '/catal

In [25]:
# 데이터프레임 및 리스트 초기화
상품DB = list()
상품cols = ['카테고리','브랜드', '상품명', '정가', '할인율', '할인가', '최저가', '평균 리뷰 점수', '리뷰 개수', '배송', '링크']
Zigzag_product = pd.DataFrame(columns=상품cols)


for link in 의류_links:
    try:
        options = Options()
        options.add_argument('--headless')  # headless mode
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        driver.get(full_url)
        
        # 페이지가 완전히 로드될 때까지 대기
        wait = WebDriverWait(driver, 10)
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, 'body')))  # 기본적인 요소 대기
        time.sleep(2)  # 추가 대기

        # 상품 페이지 HTML 파싱
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        # 디버깅: HTML 일부 출력
        
        print(f"{의류_links.index(link) + 1}번째 상품 처리중..")
        print(f"처리 중인 URL: {full_url}")
        

        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False

        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                # @type이 Product인 경우 확인
                if json_data.get("@type") == "Product":
                    found_product = True
                    ## 카테고리
                    category = '의류'
                    
                    ## 브랜드명
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    ## 상품명
                    product = json_data.get("name")
                    
                    ## 정가
                    정가 = item_page.select_one('.css-14j45be')
                    정가 = int(정가.text.replace(',', '')) if 정가 else "N/A" 
                    ## 할인율
                    할인율 = item_page.select_one('.css-1fwo2a0')
                    할인율 = int(할인율.text.replace('%', '')) * 0.01 if 할인율 else "N/A" 
                    ## 할인가
                    할인가 = item_page.select_one('.css-1dpat39')
                    할인가 = int(할인가.text.replace(',', '')) if 할인가 else "N/A"      
                    ## 최저가
                    최저가 = item_page.select_one('.css-no59fe')
                    최저가 = int(최저가.text.replace(',', '')) if 최저가 else "N/A" 
                    
                    ## 평균 평점
                    review_avgrating = json_data.get("aggregateRating", {}).get("ratingValue", "N/A")
                    ## 리뷰 개수
                    review_count = json_data.get("aggregateRating", {}).get("reviewCount", "N/A")  
                            
                    ## 배송형태
                    배송 = item_page.select_one('.css-1hh1st0')
                    배송 = 배송.text if 배송 else "직진배송" 
                    
                    ## 링크  
                    
                    상품DB.append([category, brand, product, 정가, 할인율, 할인가, 최저가, review_avgrating, review_count, 배송, full_url])
                    break
                
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        continue
    
    # 작업 완료 후 드라이버 종료
    driver.quit()


# 데이터 프레임 변환
Zigzag_product = pd.DataFrame(상품DB, columns=상품cols)

# csv에 저장
Zigzag_product.to_csv("Zigzag_의류_product.csv", index=False, encoding="utf-8-sig")

# 출력 확인
Zigzag_product

1번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/130108350
2번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/147921597
3번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/113041967
4번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/128943162
5번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/129147932
6번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/113067237
7번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/120508523
8번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/132082569
9번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/159140443
10번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/128434844
11번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/137691186
12번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/111005399
13번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/110294858
14번째 상품 처리중..
처리 중인 URL: https://zigzag.kr/catalog/products/123449569
15번째 상품 처리중..
처리 중인 URL: http

,카테고리,브랜드,상품명,정가,할인율,할인가,최저가,평균 리뷰 점수,리뷰 개수,배송,링크
0,의류,퍼빗,퍼빗 [랩티1등🏆/누적5만장🩰/모델전색소장💗/평점4.8극찬💭/기모ver.] 단독제작...,43200,0.31,37300,29840,4.8,4509,일반배송무료배송,https://zigzag.kr/catalog/products/130108350
1,의류,리얼코코,리얼코코 [단독] [봄신상🌸/브이넥&amp;라운드넥/made] 시오 니트 - 20 ...,51000,0.57,24630,22160,4.8,16998,직진배송,https://zigzag.kr/catalog/products/147921597
2,의류,고고싱,고고싱 [역대급53만장🏆11컬러/부츠컷슬랙스🌷봄팬츠] 반하이 모찌슬랙스 #세미부츠컷...,34900,0.36,24800,22320,4.7,77846,직진배송,https://zigzag.kr/catalog/products/113041967
3,의류,데일리쥬,데일리쥬 [단독][1만장판매/MADE] 라스카 레이어드 텐셀 라운드 티셔츠 (워머옵...,30000,0.36,21370,19230,4.7,5248,직진배송,https://zigzag.kr/catalog/products/128943162
4,의류,마이온샵,마이온샵 [BEST][누적2만장돌파🥇][4타입] #MADE 메로트 골지 니트티 (4...,43000,0.42,31160,24920,4.7,3215,직진배송,https://zigzag.kr/catalog/products/129147932
...,...,...,...,...,...,...,...,...,...,...,...
145,의류,디어먼트,디어먼트 [단독] 🌱[3천장돌파/크롭가디건/브이넥니트][직진배송][MNLB] 모어 ...,43000,0.48,26400,22400,4.8,500,직진배송,https://zigzag.kr/catalog/products/134887166
146,의류,에드모어,에드모어 [🌷7천장돌파/오버핏자켓/블레이저][에드모어] mjk392 루엔 베이직 자켓,97500,0.45,67500,54000,4.8,1332,직진배송,https://zigzag.kr/catalog/products/127789369
147,의류,니어웨어,니어웨어 [봄팬츠🌷 주문폭주🔥 당일발송🚚] 언유즈얼리 리드 와이드 코튼 팬츠 (2c...,35200,0.33,26280,23650,4.8,1042,직진배송,https://zigzag.kr/catalog/products/142227533
148,의류,나이니,나이니 [단독제작컬러추가!!/당일발송/누적2만장/승모근부유방커버/유교걸필수] 크래커...,15900,0.19,14310,12870,4.8,4915,일반배송무료배송,https://zigzag.kr/catalog/products/142481894


📌 결측(N/A)의 경우 직접 넣어야 함..

### 3. 리뷰 크롤링하기

In [1]:
import pickle
# 불러오기 (링크 가져오는 코드. 이 코드를 돌리면 위에 코드는 다시 안 돌려도 됨)
with open("의류_links.pkl","rb") as f:
    의류_links = pickle.load(f)
    
의류_links

['/catalog/products/130108350',
 '/catalog/products/147921597',
 '/catalog/products/113041967',
 '/catalog/products/128943162',
 '/catalog/products/129147932',
 '/catalog/products/113067237',
 '/catalog/products/120508523',
 '/catalog/products/132082569',
 '/catalog/products/159140443',
 '/catalog/products/128434844',
 '/catalog/products/137691186',
 '/catalog/products/111005399',
 '/catalog/products/110294858',
 '/catalog/products/123449569',
 '/catalog/products/141913686',
 '/catalog/products/136660836',
 '/catalog/products/129076367',
 '/catalog/products/109956114',
 '/catalog/products/150012796',
 '/catalog/products/114744900',
 '/catalog/products/114744915',
 '/catalog/products/108649128',
 '/catalog/products/101062215',
 '/catalog/products/137506712',
 '/catalog/products/133446788',
 '/catalog/products/115154949',
 '/catalog/products/135330479',
 '/catalog/products/133185968',
 '/catalog/products/127229360',
 '/catalog/products/127735100',
 '/catalog/products/140288768',
 '/catal

In [5]:
# 스크롤링 함수 (리뷰 로드 개선)
def scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=2):
    last_review_count = 0
    attempts = 0
    
    while attempts < max_attempts:
        # 페이지 끝까지 스크롤
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(2)  # 대기 시간 2초
        
        # 현재 로드된 리뷰 수 확인
        review_page = BeautifulSoup(driver.page_source, 'html.parser')
        review_elements = review_page.select("div[data-review-feed-index]")
        current_review_count = len(review_elements)
        print(f"스크롤 시도 {attempts + 1} - 현재 로드된 리뷰 개수: {current_review_count}")
        
        # 목표 리뷰 수 도달 시 종료
        if current_review_count >= desired_reviews:
            print(f"목표 리뷰 수 {desired_reviews} 도달")
            break
        
        # 리뷰 개수가 늘어나지 않으면 카운트 증가
        if current_review_count == last_review_count:
            attempts += 1
        else:
            attempts = 0
        
        last_review_count = current_review_count
    
    if attempts >= max_attempts:
        print(f"최대 시도 횟수 {max_attempts} 도달 - 더 이상 로드되지 않음")
    
    return BeautifulSoup(driver.page_source, 'html.parser')

### 0번째 ~ 9번째 상품 리뷰 크롤링

In [29]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(0, 10)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()

# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


# 데이터프레임 확인
Zigzag_review


현재 0번째 상품을 처리중입니다.
상품 정보를 찾을 수 없음
처리 중인 페이지: https://zigzag.kr/review/list/130108350
스크롤 시도 1 - 현재 로드된 리뷰 개수: 5
스크롤 시도 1 - 현재 로드된 리뷰 개수: 10
스크롤 시도 1 - 현재 로드된 리뷰 개수: 15
스크롤 시도 1 - 현재 로드된 리뷰 개수: 20
스크롤 시도 1 - 현재 로드된 리뷰 개수: 25
스크롤 시도 1 - 현재 로드된 리뷰 개수: 30
스크롤 시도 1 - 현재 로드된 리뷰 개수: 35
스크롤 시도 1 - 현재 로드된 리뷰 개수: 40
스크롤 시도 1 - 현재 로드된 리뷰 개수: 45
스크롤 시도 1 - 현재 로드된 리뷰 개수: 50
스크롤 시도 1 - 현재 로드된 리뷰 개수: 55
스크롤 시도 1 - 현재 로드된 리뷰 개수: 60
스크롤 시도 1 - 현재 로드된 리뷰 개수: 65
스크롤 시도 1 - 현재 로드된 리뷰 개수: 70
스크롤 시도 1 - 현재 로드된 리뷰 개수: 75
스크롤 시도 1 - 현재 로드된 리뷰 개수: 80
스크롤 시도 1 - 현재 로드된 리뷰 개수: 85
스크롤 시도 1 - 현재 로드된 리뷰 개수: 90
스크롤 시도 1 - 현재 로드된 리뷰 개수: 95
스크롤 시도 1 - 현재 로드된 리뷰 개수: 100
스크롤 시도 1 - 현재 로드된 리뷰 개수: 105
스크롤 시도 1 - 현재 로드된 리뷰 개수: 110
스크롤 시도 1 - 현재 로드된 리뷰 개수: 115
스크롤 시도 1 - 현재 로드된 리뷰 개수: 120
스크롤 시도 1 - 현재 로드된 리뷰 개수: 125
스크롤 시도 1 - 현재 로드된 리뷰 개수: 130
스크롤 시도 1 - 현재 로드된 리뷰 개수: 135
스크롤 시도 1 - 현재 로드된 리뷰 개수: 140
스크롤 시도 1 - 현재 로드된 리뷰 개수: 145
스크롤 시도 1 - 현재 로드된 리뷰 개수: 150
스크롤 시도 1 - 현재 로드된 리뷰 개수: 155
스크롤 시도 1 - 현재 로드된 리뷰 개수: 160
스크롤 시도

,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,N/A,N/A,솔링이,25.02.01,-3kg블랙,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm42kg상의 44,5,[158/42kg)\n\n이 옷이 유독 슬림해보이는 거 같아요 ㅎㅎ 여리여리한 느낌...,직진,있음,130108350
1,의류,N/A,N/A,콩콩땅,25.01.30,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm50kg상의 44,5,"⭐️ 스펙 : 158cm/50kg , 상체 보다 하체 발달형 \n\n⭐️ 핏, 사이...",직진,있음,130108350
2,의류,N/A,N/A,미소세계,25.01.26,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm53kg상의 55,5,발레코어룩의 정석인 아이템 발견했네여!!!ㅎㅎ 완전 여리여리해보이는 효과가 있어욤ㅠ...,일반,있음,130108350
3,의류,N/A,N/A,도귱,25.01.24,발렛베이지,생각보다 커요,보통이에요,화면과 비슷해요,150cm38kg상의 44,5,🩷핏\n너무 이뻐요!! 예전에 이런 디자인의 티셔츠 구매했다가 저한테 너무 크고 주...,직진,있음,130108350
4,의류,N/A,N/A,벚나무,25.01.23,발렛베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,169cm51kg상의 55,5,💜169cm/51kg/235/75D\n\n최근 산 상의중에 제일 맘에 듭니다\n\n...,직진,있음,130108350
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3897,의류,10가트,10가트 [봄신상][✨3만장판매/자체제작][반팔티/레이어드티] 오버핏 무지 반팔 면...,WRvIeqUO,24.12.08,크림FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,164cm58kg상의 55,5,티셔츠 디자인이 너무 예쁘게 잘 나온것 같아요 색깔도 너무 예쁩니다 사이트에서 본 ...,직진,없음,128434844
3898,의류,10가트,10가트 [봄신상][✨3만장판매/자체제작][반팔티/레이어드티] 오버핏 무지 반팔 면...,아라리yo,24.11.21,롱버전-아이보리FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,164cm66kg상의 66,5,"아무 때나 엄청 잘입고있어요 단독으로도 잘입어지고, 레이어드로도 잘입어져요. 소재도...",직진,있음,128434844
3899,의류,10가트,10가트 [봄신상][✨3만장판매/자체제작][반팔티/레이어드티] 오버핏 무지 반팔 면...,KgZjhhGU,24.11.17,롱버전-아이보리FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm45kg상의 44,5,레깅스 위에 입으려고 구매했어요\n딱 원하는 핏이라서 너무 만족스러워요.\n건조기한...,일반,있음,128434844
3900,의류,10가트,10가트 [봄신상][✨3만장판매/자체제작][반팔티/레이어드티] 오버핏 무지 반팔 면...,vsXiGTDn,24.10.24,흰색FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,163cm70kg상의 66,5,제주 여행가서 안에 입을 티가 필요하여 주문하였습니다! 남방이나 자켓에 입기 좋아요...,직진,있음,128434844


In [ ]:
print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")

### 10번째 ~ 19번째 상품 리뷰 크롤링

In [40]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(10, 20)  ## -> 10~19


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


# 데이터프레임 확인
Zigzag_review

,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,어텀,어텀 [made]베럴와이드팬츠,소금빵v,25.03.05,ivoryl,생각보다 커요,아주 만족해요,화면과 비슷해요,156cm59kg하의 26,5,총평 - ⭐️⭐️⭐️⭐️⭐️\n\n✔️구매 동기 - 이제 슬슬 봄이라 봄에 입을 후...,일반,있음,137691186
1,의류,어텀,어텀 [made]베럴와이드팬츠,유톰,25.03.10,beiges,생각보다 커요,아주 만족해요,화면과 비슷해요,170cm50kg하의 26,5,굽낮은 플랫을 신으면 바닥청소 하고 다니기는하는데 핏이 너무 예쁘고 착용감이 편해서...,일반,있음,137691186
2,의류,어텀,어텀 [made]베럴와이드팬츠,g5lFBW2K,25.03.08,ivorys,정사이즈예요,보통이에요,화면과 비슷해요,162cm52kg하의 26,4,쇼핑몰에서 바지 오랜만에 삿는데 예버서다행이에요!\n바지기장이 길다는거는 후기를 보...,일반,있음,137691186
3,의류,어텀,어텀 [made]베럴와이드팬츠,pH6TtdJn,25.03.07,ivorym,생각보다 커요,아주 만족해요,화면보다 밝아요,164cm58kg하의 29,5,사진보다 밝은 화이트인데 생화이트는 아니고 크림화이트 느낌이에요. 속옷 안비치는 도...,일반,있음,137691186
4,의류,어텀,어텀 [made]베럴와이드팬츠,ym4SwJtP,25.03.06,beigem,생각보다 커요,아주 만족해요,화면과 비슷해요,165cm57kg,5,허리가 헐렁하긴 한데 길이가 딱이라… 허리 찝어 입어도 상관없는 디자인이기도 해서 ...,일반,있음,137691186
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4265,의류,파스텔타임,파스텔타임 봄🌸new생지 추가/9만장 돌파🖤[MADE]탄탄 진청 세미 부츠컷 하이웨...,1mED6XYz,24.04.02,데일리중청S,정사이즈예요,아주 만족해요,화면과 비슷해요,163cm47kg상의 55하의 27발 240mm,5,데일리중청이랑 그레이쉬 비교샷이에요 그레이쉬는 말그대로 회색빛이라 차분하게 입기 좋...,직진,있음,114744900
4266,의류,파스텔타임,파스텔타임 봄🌸new생지 추가/9만장 돌파🖤[MADE]탄탄 진청 세미 부츠컷 하이웨...,RrQFH5ms,24.04.01,데일리중청S,정사이즈예요,아주 만족해요,화면과 비슷해요,156cm42kg상의 44하의 26발 225mm,5,역시 파스텔타임 바지맛집이네요..😍 핏은 너무 너무 이쁜데 제가 키가 작아서... ...,일반,있음,114744900
4267,의류,파스텔타임,파스텔타임 봄🌸new생지 추가/9만장 돌파🖤[MADE]탄탄 진청 세미 부츠컷 하이웨...,q1q4Hu7z,24.03.31,데일리중청M,작아요,아주 만족해요,화면과 비슷해요,160cm53kg상의 66하의 27발 235mm,5,다른 색도 되게 잘 입고 있어서 봄도 오고 하니까 밝은색으로 샀어요~근데 제가 입던...,일반,있음,114744900
4268,의류,파스텔타임,파스텔타임 봄🌸new생지 추가/9만장 돌파🖤[MADE]탄탄 진청 세미 부츠컷 하이웨...,ggr0***,24.03.31,그레이쉬L,정사이즈예요,아주 만족해요,화면보다 어두워요,165cm59kg상의 55하의 28발 245mm,5,"약간 그레이, 애쉬계열의 청바지를 사고싶어서 색상을 유심히 보고 샀어요.\n이미 리...",직진,있음,114744900


In [52]:
print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")

[상품별 크롤링된 리뷰 개수]
상품ID
109956114    165
110294858    500
111005399    500
114744900    500
123449569    260
129076367    500
136660836    500
137691186    500
141913686    345
150012796    500
dtype: int64

[상품별 리뷰 평균 점수]
              리뷰 점수
상품ID               
109956114  4.769697
110294858  4.768000
111005399  4.734000
114744900  4.816000
123449569  4.826923
129076367  4.894000
136660836  4.862000
137691186  4.828000
141913686  4.756522
150012796  4.906000


### 20~29번째

In [6]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(20, 30)  ## -> 20~29


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
101062215    500
108649128    500
114744915    500
115154949    500
127229360    500
127735100    500
133185968    495
133446788    500
135330479    500
137506712    147
dtype: int64

[상품별 리뷰 평균 점수]
             리뷰 점수
상품ID              
101062215  4.85200
108649128  4.76200
114744915  4.62000
115154949  4.56600
127229360  4.80600
127735100  4.81400
133185968  4.80000
133446788  4.85000
135330479  4.80000
137506712  4.70068



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,파스텔타임,파스텔타임 🤍흑청 컬러 추가/6천장 돌파🤍[MADE]세미 부츠컷 와이드 pt(하객룩...,hUUWV4yc,24.07.20,중청(네츄럴핏)추가기장 S,정사이즈예요,아주 만족해요,화면과 비슷해요,164cm53kg상의 66하의 26발 240mm,5,진짜 미쳣습니다… 하 사실 처음에는 m시켰거든요 저보다 마른 언니들이 s너무 작다고...,직진,있음,114744915
1,의류,파스텔타임,파스텔타임 🤍흑청 컬러 추가/6천장 돌파🤍[MADE]세미 부츠컷 와이드 pt(하객룩...,QWz754R3,24.07.12,중청(네츄럴핏)추가기장 L,정사이즈예요,아주 만족해요,화면과 비슷해요,164cm57kg상의 55하의 29발 245mm,5,"하체비만이고 특히 앞벅지, 승마살 심한 허리얇고 대퇴골 넓고 엉덩이 큰타입인데 입자...",직진,있음,114744915
2,의류,파스텔타임,파스텔타임 🤍흑청 컬러 추가/6천장 돌파🤍[MADE]세미 부츠컷 와이드 pt(하객룩...,yeij***,24.06.30,중청(네츄럴핏)추가기장 M,정사이즈예요,아주 만족해요,화면과 비슷해요,163cm51kg상의 55하의 27발 235mm,5,인생 청바지 등극할 지도.. 핏 진짜 너무 예쁨\n허리에서 골반까지의 라인이 진짜 ...,직진,있음,114744915
3,의류,파스텔타임,파스텔타임 🤍흑청 컬러 추가/6천장 돌파🤍[MADE]세미 부츠컷 와이드 pt(하객룩...,6w5tf5eJ,25.01.03,흑청기본기장 S,정사이즈예요,보통이에요,화면과 비슷해요,159cm48kg상의 44하의 26발 240mm,5,M으로 샀다가 s로 다시 구매했습니다!\n사실 후기 보고 사이즈 고민을 정말정말 많...,직진,있음,114744915
4,의류,파스텔타임,파스텔타임 🤍흑청 컬러 추가/6천장 돌파🤍[MADE]세미 부츠컷 와이드 pt(하객룩...,IWv1q4kp,23.07.27,진청S,정사이즈예요,보통이에요,화면과 비슷해요,165cm47kg상의 55하의 25발 250mm,4,어쩌다 이 바지에 꽂혔는데 다들 너무 작다해서 엠을 주문했는데 너무 크더라고요? 그...,직진,있음,114744915
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4637,의류,데일리쥬,데일리쥬 [단독][6만장판매/MADE] 페라티 베이직 가디건 (긴팔니트추가)(봄신상),TgmjEx38,24.11.19,Blue(블루/라운드넥),정사이즈예요,보통이에요,화면과 비슷해요,N/A,5,색이 선명해서 얼굴이 밝아보여요\n근데 재질은 얇아서 간절기에 잠깐 입기 좋은 것 ...,일반,있음,127735100
4638,의류,데일리쥬,데일리쥬 [단독][6만장판매/MADE] 페라티 베이직 가디건 (긴팔니트추가)(봄신상),lSFgFUBh,24.11.19,Perati Pink(페라티핑크/라운드넥),생각보다 커요,아주 만족해요,화면과 비슷해요,157cm57kg상의 66,5,색깔도 너무 이쁘고 생각보다 사이즈도 딱 이쁘게 커서 너무 좋아용 핫핑크 좋아하시는...,직진,있음,127735100
4639,의류,데일리쥬,데일리쥬 [단독][6만장판매/MADE] 페라티 베이직 가디건 (긴팔니트추가)(봄신상),2d4o2ltg,24.11.18,Perati Pink(페라티핑크/브이넥),생각보다 커요,아주 만족해요,화면과 비슷해요,172cm62kg상의 66,5,색이 아주 쨍하고 화면상 제품 컬러랑 똑같아요. 여행가서 입었는데 사진에도 잘 담겨...,직진,있음,127735100
4640,의류,데일리쥬,데일리쥬 [단독][6만장판매/MADE] 페라티 베이직 가디건 (긴팔니트추가)(봄신상),wltn****,24.11.18,Black(블랙/브이넥),정사이즈예요,보통이에요,화면과 비슷해요,159cm52kg상의 55,4,기장이나 핏이 원하던 서타일이라 맘에 들어요.\n가격대비 퀄리티가 높은 것 같고 \...,직진,있음,127735100


### 30~39번째 상품

In [7]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(30, 40)  ## -> 30~39


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
119829258    500
126244321    500
127789113    500
128321624    410
134884542    500
135005605    500
135314786    500
136095576    500
140288768    500
141360158    500
dtype: int64

[상품별 리뷰 평균 점수]
             리뷰 점수
상품ID              
119829258  4.80400
126244321  4.90600
127789113  4.83600
128321624  4.77561
134884542  4.78600
135005605  4.85400
135314786  4.85800
136095576  4.83400
140288768  4.82000
141360158  4.86400



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,리썸데이,리썸데이 [단독][🦢스커트.BEST💗] 모덴 롱 스커트 4color (벨트set) ...,헬생아,25.02.10,크림S,정사이즈예요,보통이에요,화면과 비슷해요,159cm51kg하의 27,5,"일단 저는 158/51 체지방률 24% 하비인데용,,🫠\n하비+키작아서 롱치마 안어...",직진,있음,140288768
1,의류,리썸데이,리썸데이 [단독][🦢스커트.BEST💗] 모덴 롱 스커트 4color (벨트set) ...,IlilIl,24.08.18,블랙M,정사이즈예요,아주 만족해요,화면과 비슷해요,N/A,5,"허리 얇고 골반, 엉덩이 있으신분들한테 추천드려요!\n벨트도 있어서 허리 맞게 착용...",직진,있음,140288768
2,의류,리썸데이,리썸데이 [단독][🦢스커트.BEST💗] 모덴 롱 스커트 4color (벨트set) ...,nsy5***,24.07.05,차콜S,정사이즈예요,아주 만족해요,화면과 비슷해요,167cm50kg하의 26,5,"치마 너무 이뻐요!! 길이감도 아주 적당하고, H라인이지만 뒤에가 트여있어서 전혀 ...",직진,있음,140288768
3,의류,리썸데이,리썸데이 [단독][🦢스커트.BEST💗] 모덴 롱 스커트 4color (벨트set) ...,zKHc0MmE,24.07.03,블랙S,정사이즈예요,아주 만족해요,화면과 비슷해요,161cm51kg하의 27,5,결혼식에 갈 일이 생겨 급하게 주문했어요.\n배송 빨리와서 좋았고 생각보다 크게 나...,직진,있음,140288768
4,의류,리썸데이,리썸데이 [단독][🦢스커트.BEST💗] 모덴 롱 스커트 4color (벨트set) ...,1gRKooM1,25.03.05,베이지(모델착용)M,정사이즈예요,보통이에요,화면과 비슷해요,163cm58kg,5,벨트가 같이 있어서 훨씬 고급스러워 보이는 옷이에요. 이 옷 사서 입을 때까지만 해...,직진,있음,140288768
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4905,의류,디어먼트,디어먼트 [단독] 🌱[6만장돌파/브이넥블라우스/하객룩][직진배송][MNLB] 퍼퓸 ...,tHilsFHd,24.10.21,아이보리(반팔ver.)FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,171cm58kg상의 66,5,여리여리 하면서도 깔끔하고 단정해서 이쁜 것 같아요 !\n끈이 있어서 뒤로 묶어주면...,직진,있음,134884542
4906,의류,디어먼트,디어먼트 [단독] 🌱[6만장돌파/브이넥블라우스/하객룩][직진배송][MNLB] 퍼퓸 ...,TQYeV8Zg,24.10.21,블랙FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,163cm62kg상의 66,5,받자마자 너무 예뻐서 아이보리도 사고 싶어질 정도였어요! 후기가 좋은데는 이유가 있...,직진,있음,134884542
4907,의류,디어먼트,디어먼트 [단독] 🌱[6만장돌파/브이넥블라우스/하객룩][직진배송][MNLB] 퍼퓸 ...,owE001AA,24.10.21,아이보리FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm49kg상의 44,5,"하객룩으로 급하게 구매해서 입었었는데, 마감도 좋고 퀄리티도 좋고 입었을때 핏도 너...",직진,있음,134884542
4908,의류,디어먼트,디어먼트 [단독] 🌱[6만장돌파/브이넥블라우스/하객룩][직진배송][MNLB] 퍼퓸 ...,bFcDZa40,24.10.20,블랙FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,161cm62kg상의 55,4,옷 너무 예뻐요 예쁜데\n단추 채우는게 진짜 헬이에요 🥵\n손톱 조금이라도 길거나 ...,일반,없음,134884542


### 40~49번째 상품

In [9]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(40, 50)  ## -> 30~39


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
127985991    500
128400284    500
128487064    500
128715519    500
135006673    500
135142498    500
135275623    500
135316926    500
138377569    500
142355088    500
dtype: int64

[상품별 리뷰 평균 점수]
           리뷰 점수
상품ID            
127985991  4.772
128400284  4.834
128487064  4.798
128715519  4.836
135006673  4.832
135142498  4.820
135275623  4.892
135316926  4.818
138377569  4.772
142355088  4.834



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,N/A,N/A,와사비콩,24.08.30,아이보리M,정사이즈예요,아주 만족해요,화면과 비슷해요,168cm52kg하의 27,5,사고 나서 진짜 뽕 뽑게 잘 입었어요…✨ 제가 웨이브 체형이라 요런 느낌의 스커트 ...,일반,있음,138377569
1,의류,N/A,N/A,pDCmk9UL,24.07.10,베이지M,정사이즈예요,아주 만족해요,화면과 비슷해요,161cm58kg하의 27,5,레인부츠랑 같이입을 기장의 치마가 필요해서 급하게 주문했는데 역시 직진배송 진짜 빨...,직진,있음,138377569
2,의류,N/A,N/A,fv1Jij3q,24.07.05,아이보리L,정사이즈예요,보통이에요,화면과 비슷해요,163cm60kg하의 28,4,마감처리가 살짝 부족합니다 \n지퍼가 저기서 잘 안올라가요 너무 뻑뻑해요\n골반이랑...,일반,있음,138377569
3,의류,N/A,N/A,syk1***,24.06.26,블랙S,정사이즈예요,아주 만족해요,화면과 비슷해요,N/A,5,휘뚜루 마뚜루 잘 입고 다닙니다!\n안에 속바지 있어서 넘 편하구요\n저는 허리가 ...,직진,있음,138377569
4,의류,N/A,N/A,dk02***,24.06.13,아이보리S,정사이즈예요,아주 만족해요,화면과 비슷해요,161cm46kg하의 26,5,🏷️ 착용감\n속바지 안감이라 착용감은 아주 편했어요 :)\n이런 플레어 라인의 스...,직진,있음,138377569
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,의류,슬로우앤드,슬로우앤드 [매일입는면바지/5천장돌파] #SLOWMADE. 투데이 와이드 코튼팬츠 ...,VNAwvF2U,23.12.03,아이보리S,생각보다 커요,아주 만족해요,화면과 비슷해요,163cm62kg하의 28,5,보통은 하의 M 사이즈를 입는데 이건 S도 잘 맞아요 ㅎㅎ 하비라 허벅지에 맞춰 입...,직진,있음,127985991
4996,의류,슬로우앤드,슬로우앤드 [매일입는면바지/5천장돌파] #SLOWMADE. 투데이 와이드 코튼팬츠 ...,4WFfcoqU,23.11.27,블랙S,생각보다 커요,아주 만족해요,화면과 비슷해요,162cm58kg하의 27,5,"바지 정말 정말 편하고, 너무 잘 입고 다녀요! 생각보다 큰데, 허벅지가 있는 편이...",직진,있음,127985991
4997,의류,슬로우앤드,슬로우앤드 [매일입는면바지/5천장돌파] #SLOWMADE. 투데이 와이드 코튼팬츠 ...,g01GCidH,23.11.22,블랙XS,생각보다 커요,아주 만족해요,화면과 비슷해요,160cm51kg하의 27,5,여름버전 구매해서 잘입구잇었어요 ㅎㅎ 이제 살짝 두꺼운 바지가 필요해서 재구매 했습...,직진,있음,127985991
4998,의류,슬로우앤드,슬로우앤드 [매일입는면바지/5천장돌파] #SLOWMADE. 투데이 와이드 코튼팬츠 ...,짱구야옷말려,23.11.08,블랙XS,정사이즈예요,보통이에요,화면과 비슷해요,140cm38kg하의 23,3,허리가 저한테는 살짝 큰 정도였어요! 키가 작아서 길이 수선하는 김에 허리도 조금 ...,직진,있음,127985991


### 50~59번째 상품

In [10]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(50, 60)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
110533986    500
111410187    500
114075013    500
121351880    500
127356723    500
128102706    500
129556968    500
131506009    500
136948162    500
140796259    500
dtype: int64

[상품별 리뷰 평균 점수]
           리뷰 점수
상품ID            
110533986  4.778
111410187  4.824
114075013  4.806
121351880  4.772
127356723  4.834
128102706  4.824
129556968  4.768
131506009  4.818
136948162  4.778
140796259  4.826



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,슬로우앤드,슬로우앤드 [단독] [🎖️레이어드셔츠] #SLOWMADE. 어텀 더블스티치 워싱셔츠...,kime****,24.10.17,포그그레이,정사이즈예요,아주 만족해요,화면과 비슷해요,171cm60kg상의 66,5,52세 키170 몸무게 60입니다.\n나이살과 큰 키로 인해 맞는 옷 또는 예쁜 옷...,일반,있음,127356723
1,의류,슬로우앤드,슬로우앤드 [단독] [🎖️레이어드셔츠] #SLOWMADE. 어텀 더블스티치 워싱셔츠...,jjso**,25.03.08,아이보리,정사이즈예요,아주 만족해요,화면과 비슷해요,161cm47kg상의 55,5,배송 좀 걸렸어요. 하지만 기다린 보람이 있네요.\n평좋은 이유 제가 알겠어요.\n...,일반,있음,127356723
2,의류,슬로우앤드,슬로우앤드 [단독] [🎖️레이어드셔츠] #SLOWMADE. 어텀 더블스티치 워싱셔츠...,3R9iMZPO,25.03.06,버터,정사이즈예요,아주 만족해요,화면과 비슷해요,167cm55kg상의 55,5,갈웜이다? 무조건 버터 색상 사세요!! 제가 노란 셔츠를 입고 싶었는데 입으면 얼굴...,직진,있음,127356723
3,의류,슬로우앤드,슬로우앤드 [단독] [🎖️레이어드셔츠] #SLOWMADE. 어텀 더블스티치 워싱셔츠...,m7gwUaGr,25.03.05,블랙,정사이즈예요,아주 만족해요,화면과 비슷해요,166cm56kg상의 55,5,"와인색 사구 너무 만족하며 입었어서 \n검정, 베이지 추가 구매했어용\n어쩜 셔츠가...",직진,있음,127356723
4,의류,슬로우앤드,슬로우앤드 [단독] [🎖️레이어드셔츠] #SLOWMADE. 어텀 더블스티치 워싱셔츠...,m7gwUaGr,25.03.05,매트베이지,정사이즈예요,아주 만족해요,화면과 비슷해요,166cm56kg상의 55,5,"와인색 사구 너무 만족하며 입었어서 \n검정, 베이지 추가 구매했어용\n어쩜 셔츠가...",직진,있음,127356723
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,의류,무드인슬로우,무드인슬로우 [무료배송] 플레어 핀턱 코튼스커트,36QiJp0k,23.07.05,아이보리,생각보다 커요,보통이에요,화면과 비슷해요,N/A,5,조금 도톰한 편이에요~ 옷이 구겨져서 와서 다려입으면 예쁠 것 같네요!\n지그재그...,일반,없음,110533986
4996,의류,무드인슬로우,무드인슬로우 [무료배송] 플레어 핀턱 코튼스커트,RKjo6R3f,23.06.18,블랙,정사이즈예요,보통이에요,화면과 비슷해요,162cm53kg하의 28,2,배송은 빨라요 거의 하루만에 왔어요 핏은 사진이랑은 약간 다른 거 같아요 ㅠㅠ 여름...,일반,있음,110533986
4997,의류,무드인슬로우,무드인슬로우 [무료배송] 플레어 핀턱 코튼스커트,UhxMvaSN,23.05.31,아이보리,정사이즈예요,보통이에요,화면과 비슷해요,156cm49kg하의 25,3,머가 묻어잇엇어요…하지만 반품하기 귀찮으니 다시 빨아서 입어보려고요 핏은 정말 마음...,일반,있음,110533986
4998,의류,무드인슬로우,무드인슬로우 [무료배송] 플레어 핀턱 코튼스커트,6fxI18V4,23.05.22,아이보리,정사이즈예요,보통이에요,화면과 비슷해요,174cm68kg하의 29,5,생각보다 천이 탄탄해서 예쁘게 입을 수 있었어요! 키도 큰 편인데 길이도 짧지 않고...,일반,없음,110533986


### 60~69번째 상품

In [11]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(60, 70)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
100457185    500
114266966    400
128690692    500
128693699    500
129421550    500
129984888    500
135503215    500
136080843    500
145426176    500
147167067    500
dtype: int64

[상품별 리뷰 평균 점수]
           리뷰 점수
상품ID            
100457185  4.710
114266966  4.870
128690692  4.766
128693699  4.876
129421550  4.792
129984888  4.830
135503215  4.910
136080843  4.768
145426176  4.696
147167067  4.880



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,리얼코코,리얼코코 [단독] [봄신상🌸/슬림핏/누적3만장/made] 로셀 라인 셔츠 - 7 C...,sia3***,25.03.10,핑크(스트라이프ver.)FREE,정사이즈예요,보통이에요,화면과 비슷해요,173cm55kg상의 55,5,퀸카로 살아남는 법 레지나 따라입을려고 샀어요-! 일단 셔츠가 살짝 불편해서 팔이 ...,일반,있음,145426176
1,의류,리얼코코,리얼코코 [단독] [봄신상🌸/슬림핏/누적3만장/made] 로셀 라인 셔츠 - 7 C...,난나냐냐,24.12.04,아이보리F(44~66),정사이즈예요,아주 만족해요,화면보다 어두워요,160cm47kg상의 44,5,"✔️사이즈\n저는 160cm 47kg이고 상의는 보통 s, 44반 입어요!! 이 셔...",직진,있음,145426176
2,의류,리얼코코,리얼코코 [단독] [봄신상🌸/슬림핏/누적3만장/made] 로셀 라인 셔츠 - 7 C...,웨이브체형,24.09.30,다크네이비F(44~66),작아요,아주 만족해요,화면과 비슷해요,166cm56kg상의 44,4,일단 생각보다 더 작아서 놀랐습니다\n근데 그만큼 허리를 너무 잘 잡아줘서\n몸매 ...,직진,있음,145426176
3,의류,리얼코코,리얼코코 [단독] [봄신상🌸/슬림핏/누적3만장/made] 로셀 라인 셔츠 - 7 C...,ROT0G8Sw,24.09.29,뮤트핑크F(44~66),작아요,아주 만족해요,화면과 비슷해요,168cm50kg상의 44,4,"세 모녀가 입어본 체형별 후기 공유합니다.\n우선, 바스트 및 흉곽 70b이상이면 ...",직진,있음,145426176
4,의류,리얼코코,리얼코코 [단독] [봄신상🌸/슬림핏/누적3만장/made] 로셀 라인 셔츠 - 7 C...,CJOG7PsL,24.11.15,아이보리F(44~66),작아요,아주 만족해요,화면보다 밝아요,166cm56kg상의 44,5,* 재질 : 단단하고 스판도 약간 있어서 입기에 편합니다.\n비침이 약간 있지만 그...,직진,있음,145426176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4895,의류,블랙업,블랙업 [7만장판매] [MADE] 비엔트 부츠컷 밴딩팬츠,mins******,25.02.22,그레이SLONG,정사이즈예요,아주 만족해요,화면과 비슷해요,168cm51kg,5,바지가 참 예뻐요^^ 보풀이 잘 안 일어나는 소재라 조음\n다리 길어보이고 무난해서...,일반,있음,128693699
4896,의류,블랙업,블랙업 [7만장판매] [MADE] 비엔트 부츠컷 밴딩팬츠,A1k7cSdL,25.02.22,그레이SSHORT,정사이즈예요,아주 만족해요,화면과 비슷해요,162cm51kg하의 27,5,허벅지쪽이 타이트하고 종아리는 널널한 옷입니다\n타이트한거 싫으신 분들은 한사이즈 ...,일반,있음,128693699
4897,의류,블랙업,블랙업 [7만장판매] [MADE] 비엔트 부츠컷 밴딩팬츠,U1jpZw5Z,25.02.21,블랙SSHORT,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm45kg하의 26,5,키 162에 s/short사이즈 구매했는데 길이랑 핏이 다 너무 예쁘게 떨어져용 재...,일반,있음,128693699
4898,의류,블랙업,블랙업 [7만장판매] [MADE] 비엔트 부츠컷 밴딩팬츠,phPTB5CM,25.02.21,오트밀SSHORT,정사이즈예요,보통이에요,화면과 비슷해요,157cm49kg하의 27,5,제키에는 숏도 조금 길어요 ㅜㅠ ㅎㅎ 높은 운동화 신으면 됭것같아요! 예쁩니다 엉덩...,직진,있음,128693699


### 70~79번째 상품

In [12]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(70, 80)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
109853004    500
113059030    500
123727139    500
128076596    500
129090749    500
129150099    500
132048429    500
143576257    500
147927436    100
156434505    325
dtype: int64

[상품별 리뷰 평균 점수]
              리뷰 점수
상품ID               
109853004  4.804000
113059030  4.660000
123727139  4.878000
128076596  4.744000
129090749  4.816000
129150099  4.740000
132048429  4.874000
143576257  4.836000
147927436  4.840000
156434505  4.821538



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,프렌치오브,프렌치오브 [🌷봄하객룩/2만장돌파][MADE]로제 오프숄더 드레이프 셔링 하객 봄 ...,하늘을달리다,25.02.16,CharcoalFree,정사이즈예요,아주 만족해요,화면과 비슷해요,164cm47kg상의 55,5,별점: ⭐️⭐️⭐️⭐️⭐️\n이전에 드레이프티를 구매하고 실패한 이후로 구매를 망설...,일반,있음,132048429
1,의류,프렌치오브,프렌치오브 [🌷봄하객룩/2만장돌파][MADE]로제 오프숄더 드레이프 셔링 하객 봄 ...,QyAiEGGL,25.01.20,BlackFree,정사이즈예요,아주 만족해요,화면과 비슷해요,157cm43kg상의 33 이하,5,너무 예뻐요!!\n부드러운 재질이라 피부에 닿아도 하나도 안따가워요.\n어깨 감싸는...,직진,있음,132048429
2,의류,프렌치오브,프렌치오브 [🌷봄하객룩/2만장돌파][MADE]로제 오프숄더 드레이프 셔링 하객 봄 ...,별바라기,24.12.19,IvoryFree,정사이즈예요,아주 만족해요,화면과 비슷해요,156cm48kg상의 44,5,연말모임에 꾸꾸꾸로 입을 옷 샀어요ㅎㅎ 오프숄더 중에도 노출이 덜 한 걸로 사고 싶...,직진,있음,132048429
3,의류,프렌치오브,프렌치오브 [🌷봄하객룩/2만장돌파][MADE]로제 오프숄더 드레이프 셔링 하객 봄 ...,nCH5haeO,24.12.01,BlackFree,정사이즈예요,아주 만족해요,화면과 비슷해요,156cm53kg상의 55,5,"오프숄더,, 전에 다른 곳에서 데이트룩 시도로\n핑크색 샀다가 구석에 처박아둔 1인...",직진,있음,132048429
4,의류,프렌치오브,프렌치오브 [🌷봄하객룩/2만장돌파][MADE]로제 오프숄더 드레이프 셔링 하객 봄 ...,171.4cm,24.09.07,BlackFree,생각보다 커요,아주 만족해요,화면과 비슷해요,171cm57kg상의 55,5,아주 여성스럽고 예쁜 넥라인이 매력적인 상의 입니다!\n재질도 보들보들거려서 입고 ...,일반,있음,132048429
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4420,의류,그레이레이즈,그레이레이즈 [주문폭주🔥/인기급상승📈/셔츠+가디건] 셔츠레이어드니트가디건,EnTwZ6jW,24.01.15,네이비XL(66-77),생각보다 커요,보통이에요,화면과 비슷해요,N/A,4,오늘배송 빠르게 받았습니다 \n이번주말에 입을려고 구매했는데 괜찮은듯합니다 \n색상...,일반,있음,147927436
4421,의류,그레이레이즈,그레이레이즈 [주문폭주🔥/인기급상승📈/셔츠+가디건] 셔츠레이어드니트가디건,jung*****,23.10.20,네이비FREE(44-66),정사이즈예요,보통이에요,화면보다 어두워요,164cm55kg상의 55,5,화면에서 보면서 생각했던것보다 셔츠 색깔이 살짝 어두운 파란색에 가까워서 다소 놀랐...,일반,있음,147927436
4422,의류,그레이레이즈,그레이레이즈 [주문폭주🔥/인기급상승📈/셔츠+가디건] 셔츠레이어드니트가디건,dZeBjfD1,25.02.22,네이비XL,생각보다 커요,아주 만족해요,화면과 비슷해요,160cm55kg상의 66,5,예뽀용 쿨톤한테 딱이에용 ! \n손목부분은 약간 널널한데 팔 걷으면 고정이 안되서 ...,일반,있음,147927436
4423,의류,그레이레이즈,그레이레이즈 [주문폭주🔥/인기급상승📈/셔츠+가디건] 셔츠레이어드니트가디건,S3pzlJuM,25.02.05,블랙FREE,생각보다 커요,아주 만족해요,화면과 비슷해요,200cm 이상30kg 이하상의 55,5,팔이 길다는 리뷰가 많아서 free 사이즈로 구매했는데 잘한 것 같습니다. 키에 비...,일반,있음,147927436


### 80~89번째 상품

In [13]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(80, 90)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
100468216    500
107903853    500
113840232    500
119764839    405
123358591    500
125021529    500
128076945    500
129218193    500
132644575    500
135419359    500
dtype: int64

[상품별 리뷰 평균 점수]
              리뷰 점수
상품ID               
100468216  4.860000
107903853  4.740000
113840232  4.714000
119764839  4.775309
123358591  4.728000
125021529  4.922000
128076945  4.838000
129218193  4.692000
132644575  4.784000
135419359  4.894000



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,프렌치오브,프렌치오브 [1만장돌파/핏극찬][MADE]에펠 부츠컷 흑청 블랙 하이웨스트 슬림 레...,ttosa,25.02.21,BlackS,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm53kg상의 55하의 26발 235mm,5,워싱없는 코튼 블랙진이고 생각한 느낌 그대롭니당\n부츠컷은 자칫 핏이 촌스럽게 빠질...,직진,있음,132644575
1,의류,프렌치오브,프렌치오브 [1만장돌파/핏극찬][MADE]에펠 부츠컷 흑청 블랙 하이웨스트 슬림 레...,땡이엉이,24.12.31,BlackM,생각보다 커요,아주 만족해요,화면과 비슷해요,158cm50kg상의 44하의 27발 240mm,5,"S구매했다가 허벅지, 허리가 너므 딱 맞아서 더 날씬한 엄마 드리고 ㅋㅋ 저는 M ...",직진,있음,132644575
2,의류,프렌치오브,프렌치오브 [1만장돌파/핏극찬][MADE]에펠 부츠컷 흑청 블랙 하이웨스트 슬림 레...,제이킹,24.12.09,BlackS,생각보다 커요,아주 만족해요,화면과 비슷해요,165cm49kg하의 25,5,"개이뿌네요 ,,, 사실 부츠컷 바지 인쇼에서 샀을 때 성공한적 한번도 없어서 이번ㅇ...",일반,있음,132644575
3,의류,프렌치오브,프렌치오브 [1만장돌파/핏극찬][MADE]에펠 부츠컷 흑청 블랙 하이웨스트 슬림 레...,자유롭게살고싶다,24.12.04,BlackS,정사이즈예요,아주 만족해요,화면과 비슷해요,170cm53kg하의 29,5,사이즈는 기모 스타킹을 신고 입어도 불편하지 않게 약간 늘어나는 재질이라 착용감은 ...,일반,있음,132644575
4,의류,프렌치오브,프렌치오브 [1만장돌파/핏극찬][MADE]에펠 부츠컷 흑청 블랙 하이웨스트 슬림 레...,BEqhnIIV,24.11.14,BlackM,정사이즈예요,보통이에요,화면과 비슷해요,167cm57kg상의 66하의 28발 240mm,4,과하지 않은 딱 예쁜 부츠컷이라 핏 너무 마음에 들어요!\n착용감도 편해요. \n⭐...,직진,있음,132644575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4900,의류,조이조이,"조이조이 ❣️27만장돌파❣️[봄ver/숏,기본,롱/~2XL] 인생핏 와이드 여름 투...",aTDm1HGB,25.02.28,블랙_숏L,161cm55kg하의 28,N/A,N/A,N/A,5,N/A,직진,없음,113840232
4901,의류,조이조이,"조이조이 ❣️27만장돌파❣️[봄ver/숏,기본,롱/~2XL] 인생핏 와이드 여름 투...",hxilsyqZ,25.02.28,브라운_숏2XL,155cm71kg하의 32,N/A,N/A,N/A,5,N/A,직진,없음,113840232
4902,의류,조이조이,"조이조이 ❣️27만장돌파❣️[봄ver/숏,기본,롱/~2XL] 인생핏 와이드 여름 투...",gNZKxhkB,25.02.27,챠콜_기본M,158cm56kg하의 27,N/A,N/A,N/A,4,N/A,직진,없음,113840232
4903,의류,조이조이,"조이조이 ❣️27만장돌파❣️[봄ver/숏,기본,롱/~2XL] 인생핏 와이드 여름 투...",npcm***,25.02.27,베이지_기본S,N/A,N/A,N/A,N/A,5,N/A,직진,없음,113840232


### 90~99번째 상품

In [14]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(90, 100)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
109044865     80
116180382    500
117714599     95
124126640    500
126242716     15
128400077    500
132875046    500
140431730    500
141728678    500
148125108    500
dtype: int64

[상품별 리뷰 평균 점수]
              리뷰 점수
상품ID               
109044865  4.825000
116180382  4.858000
117714599  4.663158
124126640  4.794000
126242716  4.933333
128400077  4.844000
132875046  4.794000
140431730  4.804000
141728678  4.754000
148125108  4.906000



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,매니크,매니크 [단독] [3만장판매/MNQ MADE] 에이라 스트라이프 오버핏 셔츠,5wQVaaDO,24.12.11,Blue(블루),생각보다 커요,아주 만족해요,화면과 비슷해요,168cm58kg상의 55,5,키크고 팔이 긴편이라 주로 박시한 셔츠를 사도 총기장이나 팔기장이 원하는만큼 크지 ...,직진,있음,140431730
1,의류,매니크,매니크 [단독] [3만장판매/MNQ MADE] 에이라 스트라이프 오버핏 셔츠,Jntld,24.09.21,Blue(블루),생각보다 커요,아주 만족해요,화면과 비슷해요,164cm54kg상의 66,5,"루즈하게 떨어지는, 굉장히 만족스러운 느낌의 셔츠입니다! 초가을까지 입기에 적당한 ...",직진,있음,140431730
2,의류,매니크,매니크 [단독] [3만장판매/MNQ MADE] 에이라 스트라이프 오버핏 셔츠,호박,25.03.16,Blue(블루),정사이즈예요,보통이에요,화면과 비슷해요,177cm67kg상의 66,5,"키가 크고 어깨가 넓은 체형인데도 팔 기장이 짧지 않고, 어깨도 찡기지 않아 좋았어...",직진,있음,140431730
3,의류,매니크,매니크 [단독] [3만장판매/MNQ MADE] 에이라 스트라이프 오버핏 셔츠,syl9*****,25.01.31,Blue(블루),정사이즈예요,아주 만족해요,화면과 비슷해요,160cm52kg상의 55,5,너무 좋아요 사실 옷 자체의 퀄리티는 딱 옷 값 만큼이지만 핏이 너무 맘에 들어요 ...,직진,있음,140431730
4,의류,매니크,매니크 [단독] [3만장판매/MNQ MADE] 에이라 스트라이프 오버핏 셔츠,싸리,24.12.31,Blue(블루),정사이즈예요,보통이에요,화면과 비슷해요,164cm55kg상의 55,5,여름부터 가을겨울 이너로 휘뚤마뚤하게 입고있어요! 약간 빨수록 바래지는? 원단인거 ...,직진,있음,140431730
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3685,의류,리얼코코,리얼코코 [단독] [봄신상🌸/오버핏/made] 메이킹 스트라이프 셔츠 - 7 Col...,ZMusM4Nr,24.09.28,핑크(new ver.)F(44~66),정사이즈예요,보통이에요,화면과 비슷해요,N/A,4,키 168인데 핏은 오버핏이고 전체 기장은 괜찮은데 소매 길이가 좀 짧아서 아쉬워요...,직진,있음,141728678
3686,의류,리얼코코,리얼코코 [단독] [봄신상🌸/오버핏/made] 메이킹 스트라이프 셔츠 - 7 Col...,eunh*****,24.09.27,스카이블루(new ver.)F(44~66),작아요,보통이에요,화면과 비슷해요,164cm54kg상의 55,5,팔이 좀 긴 편이라 짧다고 느껴지네요\n마감처리같은건 잘 된게 왔어요.\n\n색상도...,직진,있음,141728678
3687,의류,리얼코코,리얼코코 [단독] [봄신상🌸/오버핏/made] 메이킹 스트라이프 셔츠 - 7 Col...,바나나땅콩,24.09.26,스카이블루(new ver.)F(44~66),정사이즈예요,보통이에요,화면과 비슷해요,163cm58kg상의 55,5,청량한 색감의 스트라이프 셔츠입니당 바스락 재질이고 두께는 보통이에요 핏은 엄청난 ...,직진,있음,141728678
3688,의류,리얼코코,리얼코코 [단독] [봄신상🌸/오버핏/made] 메이킹 스트라이프 셔츠 - 7 Col...,JYrT7JW5,24.09.25,핑크(new ver.)F(44~66),생각보다 커요,보통이에요,화면과 비슷해요,160cm45kg상의 55,5,"직진 배송으로 빠르게 오고, 또 가격대비 질 괜찮네요! 디자인이 변경이라구 해서 걱...",직진,있음,141728678


### 100~109번째 상품

In [15]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(100, 110)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
111046579    500
123050447    500
124939585    500
127021121    500
127751325    235
128145379    500
134216017    500
135621079    500
136839054    500
154238846    500
dtype: int64

[상품별 리뷰 평균 점수]
              리뷰 점수
상품ID               
111046579  4.862000
123050447  4.658000
124939585  4.850000
127021121  4.792000
127751325  4.817021
128145379  4.794000
134216017  4.808000
135621079  4.880000
136839054  4.888000
154238846  4.836000



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,크림치즈마켓,"크림치즈마켓 [후기극찬~바지안감!/5천장돌파🏆] 모어 플리츠 치마바지, 미니스커트 ...",호숭,24.11.29,블랙L,정사이즈예요,아주 만족해요,화면과 비슷해요,175cm62kg하의 30,5,브라운이랑 블랙 같이샀어요! (둘 다 너무예쁨❤️)\n\n치마살땐 아무래도 기장때문...,직진,있음,134216017
1,의류,크림치즈마켓,"크림치즈마켓 [후기극찬~바지안감!/5천장돌파🏆] 모어 플리츠 치마바지, 미니스커트 ...",호숭,24.11.29,브라운L,정사이즈예요,아주 만족해요,화면과 비슷해요,175cm62kg하의 30,5,175cm 키가 큰편인데 길이감 짧지않고 아주좋아요! \n재질이 싼느낌이 아니라 두...,직진,있음,134216017
2,의류,크림치즈마켓,"크림치즈마켓 [후기극찬~바지안감!/5천장돌파🏆] 모어 플리츠 치마바지, 미니스커트 ...",fy5wvztB,24.09.20,브라운S,정사이즈예요,아주 만족해요,화면과 비슷해요,163cm47kg하의 26,5,저는 검정색도 사기로 맘 먹었어요\n가을이라 갈색 스커트 너무 가지고싶더라구요?\n...,직진,있음,134216017
3,의류,크림치즈마켓,"크림치즈마켓 [후기극찬~바지안감!/5천장돌파🏆] 모어 플리츠 치마바지, 미니스커트 ...",율둥이둥,24.07.04,그레이M,생각보다 커요,보통이에요,화면보다 어두워요,168cm55kg하의 27,5,일단 구김이 잘 가는 재질이고\n다림질 하기가 어렵습니다!\n허리가 커서 사이즈는 ...,직진,있음,134216017
4,의류,크림치즈마켓,"크림치즈마켓 [후기극찬~바지안감!/5천장돌파🏆] 모어 플리츠 치마바지, 미니스커트 ...",sje9**,24.04.25,블랙S,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm50kg하의 26,5,사람들이 괜히 많이 산게 아니네요\n재질부터 촤르르~ 좋구 안에 바지잇어서 편함\n...,직진,있음,134216017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4730,의류,디어먼트,"디어먼트 [단독] 🌱[S,M/숏,롱/기본티/이너티][직진배송][MNLB] 하프 클...",ljy6***,24.10.17,아이보리숏M,정사이즈예요,아주 만족해요,화면과 비슷해요,167cm60kg상의 55,5,딱 예뻐요!!!\n흰색롱이 품절이라서 숏으로 샀는데 셔츠나 자켓안에 입기 딱 좋은거...,직진,있음,135621079
4731,의류,디어먼트,"디어먼트 [단독] 🌱[S,M/숏,롱/기본티/이너티][직진배송][MNLB] 하프 클...",eHVL0Ntq,24.10.16,아이보리롱M,정사이즈예요,아주 만족해요,화면과 비슷해요,167cm60kg상의 66,5,퀄리티도 적당하고 색깔도 예뻐요 한 달째 입어도 별로 늘어나지는 않았고요 다만 파인...,직진,있음,135621079
4732,의류,디어먼트,"디어먼트 [단독] 🌱[S,M/숏,롱/기본티/이너티][직진배송][MNLB] 하프 클...",dxB0D4ON,24.10.14,블랙롱M,정사이즈예요,아주 만족해요,화면과 비슷해요,163cm70kg상의 77,5,생각한 것보다 옷 괜찮아요!! 좀 아쉬웠던 건 팔 부분이 좀 짧아서 불편하기한데 그...,직진,있음,135621079
4733,의류,디어먼트,"디어먼트 [단독] 🌱[S,M/숏,롱/기본티/이너티][직진배송][MNLB] 하프 클...",PmMJFkpP,24.10.14,그레이롱S,정사이즈예요,아주 만족해요,화면과 비슷해요,158cm41kg,5,"생각보다 쫀쫀한 느낌은 아니고 살짝 널널한, 여유있는 느낌이고 생각보다 많이 파였어...",직진,있음,135621079


### 110~119번째 상품

In [17]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(110, 120)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
114747784    500
115682517    500
117859063    500
121819175    500
122124557    500
122560661    500
123651231    500
128841691    500
130938071    500
131176915    500
dtype: int64

[상품별 리뷰 평균 점수]
           리뷰 점수
상품ID            
114747784  4.748
115682517  4.704
117859063  4.872
121819175  4.854
122124557  4.788
122560661  4.784
123651231  4.848
128841691  4.748
130938071  4.786
131176915  4.834



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,나이니,나이니 [단독][new컬러추가/누적판매3만장!/추가할인/끈조절] 소녀 골지 레이스 ...,소금빵v,25.03.15,아이보리선택 안함,정사이즈예요,아주 만족해요,화면과 비슷해요,156cm59kg상의 55,5,총평 - ⭐️⭐️⭐️⭐️⭐️\n\n✔️ 구매 동기 - 남자친구랑 기념일에 여행을 가...,직진,있음,122560661
1,의류,나이니,나이니 [단독][new컬러추가/누적판매3만장!/추가할인/끈조절] 소녀 골지 레이스 ...,jjm2***,24.11.20,블랙선택안함,정사이즈예요,아주 만족해요,화면과 비슷해요,161cm58kg상의 66,5,레이어드용으로 샀는데 생각보다 야해여\n저는 단독착용은 못할것 같습니당\n레이어드해...,직진,있음,122560661
2,의류,나이니,나이니 [단독][new컬러추가/누적판매3만장!/추가할인/끈조절] 소녀 골지 레이스 ...,BYdfCbBZ,24.05.20,아이보리(i)선택안함,정사이즈예요,아주 만족해요,화면과 비슷해요,163cm56kg상의 55,5,가슴이 저는 좀 큰 편이라 여리여리 핏은 안나오지만 기여워용 근데 가슴 크신분들은 ...,직진,있음,122560661
3,의류,나이니,나이니 [단독][new컬러추가/누적판매3만장!/추가할인/끈조절] 소녀 골지 레이스 ...,5wwWuK1d,24.03.20,아이보리선택안함,작아요,아주 만족해요,화면과 비슷해요,160cm48kg상의 55,5,사실 이너로 입을거라 퀄리티는 크게 기대안했는데 레이스도 촘촘하니 예쁘고 무엇보다 ...,일반,있음,122560661
4,의류,나이니,나이니 [단독][new컬러추가/누적판매3만장!/추가할인/끈조절] 소녀 골지 레이스 ...,yMVg41BY,25.03.15,아이보리선택 안함,정사이즈예요,보통이에요,화면과 비슷해요,157cm48kg상의 44,4,요즘 레이스 나시 많이 보이길래 하나 구매해봤어요. 배송은 각져있게 예쁘게 접혀서 ...,직진,있음,122560661
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,의류,달리호텔,달리호텔 [단독] [Dali made] 아델 브이넥 랩 니트 가디건,cRfOMOG3,23.03.22,민트,정사이즈예요,아주 만족해요,화면과 비슷해요,169cm50kg상의 55,4,적당한 핏이라 예뻐요!! 얼굴 환해보이는 색상이고 날 풀리고 입으면 더 예쁠거 같아...,일반,있음,115682517
4996,의류,달리호텔,달리호텔 [단독] [Dali made] 아델 브이넥 랩 니트 가디건,0Gfdtoi3,23.03.20,레몬FREE,정사이즈예요,보통이에요,화면과 비슷해요,170cm55kg상의 55,4,기대 안 하고 샀는데 생각보다 색감이며 핏이며 너무 예뻐요ㅜㅜ\n옷이 그냥 봄 그 ...,직진,있음,115682517
4997,의류,달리호텔,달리호텔 [단독] [Dali made] 아델 브이넥 랩 니트 가디건,qoCGYg13,23.03.18,레몬FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm50kg상의 55,5,부들부들 촉감 엄청 좋아요~~\n적당한 두께감에 안에 비치지도 않고 랩이라 가슴부분...,직진,있음,115682517
4998,의류,달리호텔,달리호텔 [단독] [Dali made] 아델 브이넥 랩 니트 가디건,gusw*****,23.03.14,크림FREE,정사이즈예요,보통이에요,화면과 비슷해요,160cm50kg상의 55,5,다른 분들의 후기처럼 가슴쪽이 벌어져서 계속 신경이 쓰이기는 했지만 하나만 입어도 ...,직진,있음,115682517


### 120~129번째 상품

In [18]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(120, 130)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
126327241    500
135503370    500
135504282    500
136701133    500
138308677    500
139250095    500
139471590    455
139472616    265
139968203    500
140252243    500
dtype: int64

[상품별 리뷰 평균 점수]
              리뷰 점수
상품ID               
126327241  4.896000
135503370  4.912000
135504282  4.816000
136701133  4.666000
138308677  4.834000
139250095  4.826000
139471590  4.857143
139472616  4.890566
139968203  4.822000
140252243  4.840000



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,N/A,N/A,패딩턴패딩스탑,24.11.20,(19)Black (24FW)100,정사이즈예요,아주 만족해요,화면과 비슷해요,171cm64kg상의 66,5,후기는 남겨야겠고 당장에 입고갈곳은\n없어 최악의 코디로 남겨보는 자켓후기🫣\n매장...,직진,있음,126327241
1,의류,N/A,N/A,JsbVh8bq,25.03.17,(19)Black (24FW)090,생각보다 커요,아주 만족해요,화면과 비슷해요,155cm41kg상의 44,5,진짜 너무 만족스러워요 전부터 살지 말지 고민 오지게 하다가 여행 갈 때 가져가려고...,직진,있음,126327241
2,의류,N/A,N/A,IOhzEMvZ,25.03.08,(19)Black (24FW)095,생각보다 커요,아주 만족해요,화면과 비슷해요,157cm55kg상의 66,5,품이 넉넉해 착용감은 편한데다가 이뻐요\n가격은 저렴한데 소재는 전혀 저렴해 보이...,직진,있음,126327241
3,의류,N/A,N/A,c55PFa1s,25.03.07,(19)Black (24FW)090,정사이즈예요,아주 만족해요,화면과 비슷해요,168cm51kg상의 44,5,일본 여행 갔을 때 입으려고 샀는데 가격에 비해 퀄리티가 좋아서 놀랐습니다! 너무 ...,직진,있음,126327241
4,의류,N/A,N/A,쫄깃닭다리살,25.03.03,(19)Black (24FW)090,정사이즈예요,아주 만족해요,화면과 비슷해요,161cm48kg상의 55,5,여행용으로 급하게 구매해서 반신반의 했는데! 마음에 쏙 들었고 여행내내 잘 입고 다...,직진,있음,126327241
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4715,의류,젝시믹스,젝시믹스 데일리페더 슬림핏 숏슬리브 1+1,m9vU4W9Y,24.10.29,실리트그레이S(44~55),정사이즈예요,아주 만족해요,화면과 비슷해요,170cm55kg상의 55,5,아이스페더보다 퀄리티도 좋고 제 체형에 더 잘 어울리는 것 같아요 다른 색상도 추가...,직진,있음,135503370
4716,의류,젝시믹스,젝시믹스 데일리페더 슬림핏 숏슬리브 1+1,sJCNBc4b,24.10.23,블랙 S실리트그레이 S,정사이즈예요,보통이에요,화면과 비슷해요,N/A,4,사이즈도 딱 좋고.얇아서 땀이 금방 마를것 같아요. 그런데 뒷목쪽이 왜인지 모르게 ...,일반,있음,135503370
4717,의류,젝시믹스,젝시믹스 데일리페더 슬림핏 숏슬리브 1+1,유니쿠마,24.10.19,백아이보리 S아스텔그레이 S,정사이즈예요,아주 만족해요,화면과 비슷해요,164cm49kg상의 44,5,헬스장에서 입고 있는데 아주 가볍고 땀이 나도 티나지 않아 편하고 운동할 때 딱입니...,일반,없음,135503370
4718,의류,젝시믹스,젝시믹스 데일리페더 슬림핏 숏슬리브 1+1,dwFpikuj,24.10.15,블랙 M퍼플하트 M,정사이즈예요,보통이에요,화면과 비슷해요,160cm50kg상의 55,5,헬스할때 잘 입고 있어요 편하고 땀자국도 안 나고 세탁하면 금방 말라서 좋음요,일반,있음,135503370


### 130~139번째 상품

In [19]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(130, 140)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
121549666     15
123735073    500
131367946     10
132440480     15
133787716     85
134769425    170
134770445    500
135012008    500
135422909    500
dtype: int64

[상품별 리뷰 평균 점수]
              리뷰 점수
상품ID               
121549666  4.933333
123735073  4.818000
131367946  4.900000
132440480  4.866667
133787716  4.811765
134769425  4.794118
134770445  4.840000
135012008  4.852000
135422909  4.798000



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,케이클럽,케이클럽 [단독][자체제작/4만장돌파!/뉴컬러추가!/S-XL]#봄🌷 델루나 플리츠 ...,살때최소한시간서치,25.01.07,블랙M(27-28),정사이즈예요,보통이에요,화면과 비슷해요,164cm60kg하의 38,5,🛒 구매 계기\n출근하며 매일 매일 바지만 입고 다니는 게 단조로웠는데 치마를 입자...,직진,있음,135422909
1,의류,케이클럽,케이클럽 [단독][자체제작/4만장돌파!/뉴컬러추가!/S-XL]#봄🌷 델루나 플리츠 ...,서니아니곳서니,24.12.26,베이지L(29-30),생각보다 커요,아주 만족해요,화면과 비슷해요,166cm66kg하의 34,5,10월 일본 여행으로 급하기 직진배송으로 시켰는데 허리가 좀 컸던거 빼고는 너무 좋...,직진,있음,135422909
2,의류,케이클럽,케이클럽 [단독][자체제작/4만장돌파!/뉴컬러추가!/S-XL]#봄🌷 델루나 플리츠 ...,우리동네귀한분,25.03.12,베이지S(25-26),정사이즈예요,아주 만족해요,화면과 비슷해요,157cm57kg하의 27,5,세탁을 하고 입어서 다림질 전 입니다만\n너무너무 예쁘고 일본 여행갈때 입으려고 샀...,직진,있음,135422909
3,의류,케이클럽,케이클럽 [단독][자체제작/4만장돌파!/뉴컬러추가!/S-XL]#봄🌷 델루나 플리츠 ...,7FyTRJNh,25.03.12,아이보리M(27-28),정사이즈예요,아주 만족해요,화면과 비슷해요,163cm63kg하의 30,5,화이트 풀스커트가 대세라길래 구입해봤어요.플레어인데다 화이트라 부해보이지 않을까 걱...,직진,있음,135422909
4,의류,케이클럽,케이클럽 [단독][자체제작/4만장돌파!/뉴컬러추가!/S-XL]#봄🌷 델루나 플리츠 ...,IpHcu4yN,25.02.27,아이보리S(25-26),정사이즈예요,아주 만족해요,화면과 비슷해요,160cm48kg하의 26,5,많이 구겨져서 와서 볼품없어보이지만 주름피면 정말 예쁠 것 같아요! 아쉽긴한데 피는...,일반,있음,135422909
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2290,의류,N/A,N/A,UU7DwLEK,24.04.22,블랙FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,168cm60kg상의 66,5,면이 부들거리고 너무 편해요 입으면 진짜 날씬해 보여서 대만족입니다 요즘같은 날씨에...,일반,있음,123735073
2291,의류,N/A,N/A,Ek5YoMuI,24.04.22,블랙FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,154cm45kg상의 55,5,사이즈널럴해요\n차르르떨어지는게 편해서 자주입을것같아요\n얇아서 지금입기 딱좋네요~...,일반,있음,123735073
2292,의류,N/A,N/A,A2pHHtNY,24.04.21,네이비FREE,정사이즈예요,아주 만족해요,화면과 비슷해요,160cm49kg상의 55,5,촤르륵 떨어지는 핏이라 넘 여리여리 합니당🤍 마감이 진짜 좋아요!! 요즘 제일 잘 ...,직진,있음,123735073
2293,의류,N/A,N/A,0WXH9kJw,24.04.18,네이비FREE,정사이즈예요,보통이에요,화면보다 밝아요,166cm69kg상의 55,5,예뻐용 ㅠㅠ 핏 굿!!! 색깔 별로 쟁길라고 블랙도 구매햇져요 이 스토어에서 구매...,직진,있음,123735073


### 140~149번째 상품

In [20]:
# 데이터프레임 및 리스트 초기화
리뷰DB = list()
리뷰cols = ['카테고리', '브랜드', '상품명', '작성자', '작성 날짜', '옵션', '사이즈', '퀄리티', '색감', '정보', '리뷰 점수', '리뷰 코멘트', '배송형태', '포토리뷰', '상품ID']
Zigzag_review = pd.DataFrame(columns=리뷰cols)


## 몇 번째인지 적어주기
n = range(140, 150)


# 메인 크롤링 로직
for i in n:
    link = 의류_links[i]
    try:
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        full_url = f'https://zigzag.kr{link}'
        print(f"\n현재 {의류_links.index(link)}번째 상품을 처리중입니다.")
        driver.get(full_url)
        
        # 카테고리 설정
        category = '의류'
        
        # 상세 페이지에서 브랜드명, 상품명 추출
        item_page = BeautifulSoup(driver.page_source, 'html.parser')
        json_ld_scripts = item_page.select('script[type="application/ld+json"]')
        found_product = False
        
        for script in json_ld_scripts:
            try:
                json_data = json.loads(script.string)
                if json_data.get("@type") == "Product":
                    found_product = True
                    brand = json_data.get("brand", {}).get("name", "N/A")
                    product = json_data.get("name", "N/A")
                    break
            except json.JSONDecodeError as e:
                print(f"JSON 파싱 오류: {e}")
                continue
        
        if not found_product:
            print("상품 정보를 찾을 수 없음")
            brand, product = "N/A", "N/A"
        driver.quit()
        
        ## --------------------------------여기까지 오류 없음-------------------------------------------------------- ##
        
        # WebDriver 설정 (Headless 모드 비활성화)
        options = Options()
        # options.add_argument('--headless')  # Headless 모드 끔
        driver = webdriver.Chrome(options=options)
        product_url = link.split("/")[-1]
        full_url = f'https://zigzag.kr/review/list/{product_url}'
        print(f"처리 중인 페이지: {full_url}")
        driver.get(full_url)
        
        
        # 스크롤링으로 리뷰 로드
        review_page = scroll_to_load_reviews(driver, desired_reviews=500, max_attempts=1)   #### here : 리뷰 500개
        
        # 최대 인덱스 계산
        review_elements = review_page.select("div[data-review-feed-index]")
        if review_elements:
            max_index = max(int(elem["data-review-feed-index"]) for elem in review_elements)
        else:
            max_index = 0
        print(f"최대 리뷰 개수: {max_index + 1}")

        # 리뷰 데이터 수집
        for index in range(max_index + 1):
            review_element = review_page.select_one(f"div[data-review-feed-index='{index}']")
            if not review_element:
                print(f"리뷰 인덱스 {index} 없음")
                continue
            
            try:
                # 사용자명
                username = review_element.select_one("p.zds4_s96ru815")
                username = username.text.strip() if username else "N/A"
                
                # 작성 날짜
                date = review_element.select_one("div.css-1xqlji6 span.zds4_s96ru81d")
                date = date.text.strip() if date else "N/A"
                
                # 리뷰 점수
                review_element2 = review_element.select_one("div.css-1uhjnza.e170a1kd0")
                yellow_stars = review_element2.select("svg[style*='--zds4_dgssjf0: var(--zds-color-palette-yellow-300);']") if review_element2 else []
                rating = len(yellow_stars) if yellow_stars else "N/A"
                
                # 옵션, 사이즈, 퀄리티, 색감, 정보
                div_elements = review_element.select("div.zds4_s96ru86.zds4_s96ru819")
                odd_texts = []
                for i, div in enumerate(div_elements):
                    if i % 2 == 1:
                        text = div.text.strip() if div.text else "N/A"
                        odd_texts.append(text)
                while len(odd_texts) < 5:
                    odd_texts.append("N/A")
                
                option, size, quality, color, info = odd_texts[:5]
                
                # 배송형태
                delivery = "일반"
                if review_element.find("img", class_="css-o092jf eld8gav3"):
                    delivery = "직진"
                
                # 리뷰 내용
                review_text_element = review_element.select_one("span.zds4_s96ru813")
                review_text = review_text_element.text.strip() if review_text_element else "N/A"
                
                # 포토리뷰 여부
                photo = "없음"
                if review_element.select_one("div.css-s01evr.efs1gt61"):
                    photo = "있음"
                
                # 리뷰 데이터 저장
                리뷰DB.append([category, brand, product, username, date, option, size, quality, color, info, rating, review_text, delivery, photo, product_url])
            
            except Exception as e:
                print(f"리뷰 인덱스 {index} 데이터 추출 실패: {e}")
        
        driver.quit()

    except Exception as e:
        print(f"상품 처리 중 에러 발생: {e}")
        driver.quit()
    finally:
        driver.quit()
    clear_output(wait=True)
    
# Zigzag_review 변수에 데이터프레임 저장
Zigzag_review = pd.DataFrame(리뷰DB, columns=리뷰cols)
# Zigzag_review csv 저장
Zigzag_review.to_csv(f"Zigzag_의류_review_{n[0]}~{n[-1]}.csv", index=False, encoding='utf-8-sig')


print(f"[상품별 크롤링된 리뷰 개수]\n{Zigzag_review.groupby('상품ID').size()}")
print()
print(f"[상품별 리뷰 평균 점수]\n{Zigzag_review.groupby('상품ID').mean('리뷰점수')}")
print()

# 데이터프레임 확인
Zigzag_review

[상품별 크롤링된 리뷰 개수]
상품ID
110444086    500
112289011    500
114191662    500
122210777    500
125739098    500
127789369    500
134887166    500
142227533    500
142481894    500
150592199    500
dtype: int64

[상품별 리뷰 평균 점수]
           리뷰 점수
상품ID            
110444086  4.688
112289011  4.892
114191662  4.762
122210777  4.880
125739098  4.748
127789369  4.840
134887166  4.840
142227533  4.790
142481894  4.858
150592199  4.802



,카테고리,브랜드,상품명,작성자,작성 날짜,옵션,사이즈,퀄리티,색감,정보,리뷰 점수,리뷰 코멘트,배송형태,포토리뷰,상품ID
0,의류,젝시믹스,젝시믹스 [마지막특가] V업 3D 플러스 레깅스 1+1,오기,24.12.06,블랙 M어텀카키 M,정사이즈예요,아주 만족해요,화면과 비슷해요,168cm53kg,5,화면이랑 색감이랑 비슷했고\n카키색이 생각보다 무난해서 아무 운동복에 매치해도\n너...,일반,있음,112289011
1,의류,젝시믹스,젝시믹스 [마지막특가] V업 3D 플러스 레깅스 1+1,오기,24.09.29,카본네이비 M월넛베이지 M,정사이즈예요,아주 만족해요,화면보다 어두워요,168cm,5,애정하는 젝시믹스 레깅스 .. 🩵\n윌넛베이지 밝은 색감이라 부담스럽지 않을까 걱정...,일반,있음,112289011
2,의류,젝시믹스,젝시믹스 [마지막특가] V업 3D 플러스 레깅스 1+1,토깽스,25.03.05,블랙S(44~55),정사이즈예요,아주 만족해요,화면과 비슷해요,160cm51kg하의 25,5,사이즈 고민하다가 썬더그레이 m 블랙 s로 시켜봤는데 사진 참고해주세요! \ns 작...,직진,있음,112289011
3,의류,젝시믹스,젝시믹스 [마지막특가] V업 3D 플러스 레깅스 1+1,토깽스,25.03.05,썬더그레이M(55반~66),정사이즈예요,아주 만족해요,화면과 비슷해요,160cm51kg하의 25,5,사이즈 고민하다가 썬더그레이 m 블랙 s로 시켜봤는데 사진 참고해주세요! \ns 작...,직진,있음,112289011
4,의류,젝시믹스,젝시믹스 [마지막특가] V업 3D 플러스 레깅스 1+1,dldb******,25.02.22,다크그레이S(44~55),정사이즈예요,보통이에요,화면보다 밝아요,159cm48kg하의 26,5,"전버전보다 얇고, 신축성이 좋아진 느낌이에요. 전엔 입을때 힘들었는데 지금은 부드럽...",직진,있음,112289011
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,의류,고고싱,고고싱 [🌷봄팬츠 New브라운🤎5만2천장돌파!!] 중독팬츠ver.코튼와이드 (츄리닝...,uwtBnqC4,25.03.04,블랙ver.155L,생각보다 커요,보통이에요,화면과 비슷해요,N/A,5,너무 편해서 깔별로 쟁였습니다\n특히 블랙컬러는 아주 무난해서\n엄마도 하나 사드렸...,직진,있음,150592199
4996,의류,고고싱,고고싱 [🌷봄팬츠 New브라운🤎5만2천장돌파!!] 중독팬츠ver.코튼와이드 (츄리닝...,zqAELX1M,25.03.03,블랙ver.155L,생각보다 커요,아주 만족해요,화면과 비슷해요,159cm67kg하의 28,5,이 바지를 색깔별로 가지고 있는데 아직까지 무릎 늘어남도 없고 잘 입고 있어요!\n...,직진,없음,150592199
4997,의류,고고싱,고고싱 [🌷봄팬츠 New브라운🤎5만2천장돌파!!] 중독팬츠ver.코튼와이드 (츄리닝...,1K2ZSHnj,25.03.01,화이트ver.165M,정사이즈예요,아주 만족해요,화면과 비슷해요,162cm52kg하의 27,5,사이즈가 좀 크게 나왔는지 보통 엠사이즈 입어서 M 사이즈로 시켰는데 널널해서 많이...,직진,있음,150592199
4998,의류,고고싱,고고싱 [🌷봄팬츠 New브라운🤎5만2천장돌파!!] 중독팬츠ver.코튼와이드 (츄리닝...,밍팡이,25.03.01,블랙ver.160S,정사이즈예요,아주 만족해요,화면보다 어두워요,158cm52kg하의 28,5,키158cm인데 160버전으로 구매했어요 뱃살 많은 체형이라 허리 28인치 입는데 ...,직진,있음,150592199
